### Packages

In [8]:
%pip install openai python-dotenv


Note: you may need to restart the kernel to use updated packages.


### Environment Variables and Paths

In [10]:
import os
import re
import openai
import json
from dotenv import load_dotenv
from pathlib import Path

from openai import OpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")



if not OPENAI_API_KEY:
    raise ValueError("OpenAI API key not found. Please set it in the .env file.")

openai_client = OpenAI(api_key=OPENAI_API_KEY)

# Define the base directory
BASE_DIR = Path.cwd()

# Modules directory
MODULES_DIR = BASE_DIR / "Modules"

# Define output directory for skill maps
SKILL_MAP_DIR = BASE_DIR / "skill_map"
SKILL_MAP_DIR.mkdir(exist_ok=True)

print("Setup complete.")

NameError: name 'OPENAI_API_KEY' is not defined

### Read Markdown

In [11]:
def read_markdown(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()


### Generate Detailed Skill Map

In [12]:
def generate_skill_map(module_number, learning_goals, previous_skills=None):
    """
    Generates a detailed skill map for a module using OpenAI's API.

    :param module_number: The current module number (e.g., M1, M2).
    :param learning_goals: The basic learning goals extracted from learning_goals.md.
    :param previous_skills: Accumulated skills from previous modules.
    :return: Detailed skill map as a string.
    """
    prompt = f"""
You are an educational content creator specializing in programming courses. Based on the following basic learning goals for Module {module_number}, expand them into a comprehensive and detailed skill map. The skill map should include a general description and bullet points with detailed sub-points. Ensure that the skills are relational and sequential, building upon the skills from previous modules.

**Basic Learning Goals:**
{learning_goals}

**Previous Skills:**
{previous_skills if previous_skills else "None"}

**Detailed Skill Map:**
"""

    try:
        response = openai_client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are ChatGPT, a large language model trained by OpenAI."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,
            temperature=0.3,
        )
        detailed_skill_map = response.choices[0].message['content'].strip()
        return detailed_skill_map
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return None


###  Iterate Through Modules and Generate Skill Maps

In [13]:
from collections import OrderedDict

# Initialize an ordered dictionary to keep track of accumulated skills
accumulated_skills = OrderedDict()

# Iterate through each module in order (sorted by module number)
for module_dir in sorted(MODULES_DIR.iterdir(), key=lambda x: x.name):
    if module_dir.is_dir():
        module_number = module_dir.name  # e.g., M1, M2, etc.
        print(f"Processing {module_number}...")

        # Path to learning_goals.md
        learning_goals_path = module_dir / "learning_goals.md"
        if not learning_goals_path.exists():
            print(f"  Skipping {module_number}: learning_goals.md not found.")
            continue

        # Read basic learning goals
        basic_learning_goals = read_markdown(learning_goals_path)

        # Get accumulated skills as a string
        previous_skills = "\n".join(accumulated_skills.values()) if accumulated_skills else None

        # Generate detailed skill map
        detailed_skill_map = generate_skill_map(module_number, basic_learning_goals, previous_skills)
        if detailed_skill_map:
            print(f"  Generated skill map for {module_number}.")
            # Store in accumulated_skills
            accumulated_skills[module_number] = detailed_skill_map
        else:
            print(f"  Failed to generate skill map for {module_number}.")


NameError: name 'MODULES_DIR' is not defined

### Parse Skill Map

In [ ]:
def parse_skill_map(detailed_skill_map):
    """
    Parses the detailed skill map text into a dictionary.

    :param detailed_skill_map: The detailed skill map as a string.
    :return: Parsed skill map as a dictionary.
    """
    skill_map = {}
    
    # Split the response into lines
    lines = detailed_skill_map.split('\n')
    
    # Initialize variables
    description = []
    bullet_points = []
    sub_bullets = []
    current_section = 'description'

    for line in lines:
        stripped_line = line.strip()
        if stripped_line.startswith('- '):
            # Main bullet point
            bullet = stripped_line.lstrip('- ').strip()
            bullet_points.append(bullet)
            current_section = 'bullet'
        elif stripped_line.startswith('* '):
            # Sub-bullet point
            sub_bullet = stripped_line.lstrip('* ').strip()
            if bullet_points:
                bullet_points[-1] += f" - {sub_bullet}"
        elif stripped_line:
            if current_section == 'description':
                description.append(stripped_line)
    
    skill_map['general_description'] = ' '.join(description)
    skill_map['detailed_points'] = bullet_points
    
    return skill_map
